In [329]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers,regularizers
from sklearn.preprocessing import LabelEncoder

In [330]:
training_set = pd.read_excel('train_matches.xlsx').set_index('id')
training_set = training_set.loc[:,'city':'win_by_wickets']
training_set['toss_decision'] =  (training_set['toss_decision']=='field').astype(float)
training_set['result'] = (training_set['result']=='normal').astype(int)
for i in range(1,637):
    training_set.loc[i,'month'] = training_set.loc[i,'date'].month/12
    training_set.loc[i,'day'] = training_set.loc[i,'date'].day/30
training_set

,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,month,day
id,,,,,,,,,,,,,
1,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,1.0,1,0,Sunrisers Hyderabad,35,0,0.333333,0.166667
2,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,1.0,1,0,Rising Pune Supergiant,0,7,0.333333,0.200000
3,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,1.0,1,0,Kolkata Knight Riders,0,10,0.333333,0.233333
4,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,1.0,1,0,Kings XI Punjab,0,6,0.333333,0.266667
5,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,0.0,1,0,Royal Challengers Bangalore,15,0,0.333333,0.266667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,Raipur,2016-05-22,Delhi Daredevils,Royal Challengers Bangalore,Royal Challengers Bangalore,1.0,1,0,Royal Challengers Bangalore,0,6,0.416667,0.733333
633,Bangalore,2016-05-24,Gujarat Lions,Royal Challengers Bangalore,Royal Challengers Bangalore,1.0,1,0,Royal Challengers Bangalore,0,4,0.416667,0.800000
634,Delhi,2016-05-25,Sunrisers Hyderabad,Kolkata Knight Riders,Kolkata Knight Riders,1.0,1,0,Sunrisers Hyderabad,22,0,0.416667,0.833333


In [331]:
df = pd.read_excel('train_deliveries.xlsx').set_index(['match_id','inning'])
df

batting_team                 bowling_team  \
match_id inning                                                             
1        1               Sunrisers Hyderabad  Royal Challengers Bangalore   
         1               Sunrisers Hyderabad  Royal Challengers Bangalore   
         1               Sunrisers Hyderabad  Royal Challengers Bangalore   
         1               Sunrisers Hyderabad  Royal Challengers Bangalore   
         1               Sunrisers Hyderabad  Royal Challengers Bangalore   
...                                      ...                          ...   
636      2       Royal Challengers Bangalore          Sunrisers Hyderabad   
         2       Royal Challengers Bangalore          Sunrisers Hyderabad   
         2       Royal Challengers Bangalore          Sunrisers Hyderabad   
         2       Royal Challengers Bangalore          Sunrisers Hyderabad   
         2       Royal Challengers Bangalore          Sunrisers Hyderabad   

                 over  ball        batsman    non_striker    bowler  \
match_id inning                                                       
1        1          1     1      DA Warner       S Dhawan  TS Mills   
         1          1     2      DA Warner       S Dhawan  TS Mills   
         1          1     3      DA Warner       S Dhawan  TS Mills   
         1          1     4      DA Warner       S Dhawan  TS Mills   
         1          1     5      DA Warner       S Dhawan  TS Mills   
...               ...   ...            ...            ...       ...   
636      2         20     2    Sachin Baby      CJ Jordan   B Kumar   
         2         20     3    Sachin Baby      CJ Jordan   B Kumar   
         2         20     4  Iqbal Abdulla    Sachin Baby   B Kumar   
         2         20     5    Sachin Baby  Iqbal Abdulla   B Kumar   
         2         20     6  Iqbal Abdulla    Sachin Baby   B Kumar   

                 is_super_over  wide_runs  bye_runs  legbye_runs  noball_runs  \
match_id inning                                                                 
1        1                   0          0         0            0            0   
         1                   0          0         0            0            0   
         1                   0          0         0            0            0   
         1                   0          0         0            0            0   
         1                   0          2         0            0            0   
...                        ...        ...       ...          ...          ...   
636      2                   0          0         0            0            0   
         2                   0          0         0            0            0   
         2                   0          0         0            1            0   
         2                   0          0         0            0            0   
         2                   0          0         0            0            0   

                 penalty_runs  batsman_runs  extra_runs  total_runs  \
match_id inning                                                       
1        1                  0             0           0           0   
         1                  0             0           0           0   
         1                  0             4           0           4   
         1                  0             0           0           0   
         1                  0             0           2           2   
...                       ...           ...         ...         ...   
636      2                  0             2           0           2   
         2                  0             0           0           0   
         2                  0             0           1           1   
         2                  0             1           0           1   
         2                  0             4           0           4   

                player_dismissed dismissal_kind  fielder  
match_id inning                                           
1        1

In [332]:
for i in range(1,637):
    try:
        t1 = df.loc[i].loc[1]
        t2 = df.loc[i].loc[2]
    except:
        continue
    runs1 = sum(t1.loc[:,'total_runs'])
    runs2 = sum(t2.loc[:,'total_runs'])
    wick1 = t1.loc[:,'player_dismissed'].count()
    wick2 = t2.loc[:,'player_dismissed'].count()
    fours1 = t1.loc[t1['batsman_runs']==4,'batsman_runs'].count()
    fours2 = t2.loc[t2['batsman_runs']==4,'batsman_runs'].count()
    six1 = t1.loc[t1['batsman_runs']==6,'batsman_runs'].count()
    six2 = t2.loc[t2['batsman_runs']==6,'batsman_runs'].count()
    if training_set.loc[i,'team1']==t1.iloc[0,0]:
        training_set.loc[i,'runs1'] = runs1
        training_set.loc[i,'runs2'] = runs2
        training_set.loc[i,'wickets1'] = wick1
        training_set.loc[i,'wickets2'] = wick2
        training_set.loc[i,'fours1'] = fours1
        training_set.loc[i,'fours2'] = fours2
        training_set.loc[i,'sixes1'] = six1
        training_set.loc[i,'sixes2'] = six2
    else:
        training_set.loc[i,'runs1'] = runs2
        training_set.loc[i,'runs2'] = runs1
        training_set.loc[i,'wickets1'] = wick2
        training_set.loc[i,'wickets2'] = wick1
        training_set.loc[i,'fours1'] = fours2
        training_set.loc[i,'fours2'] = fours1
        training_set.loc[i,'sixes1'] = six2
        training_set.loc[i,'sixes2'] = six1
    training_set.loc[i,'toss_winner'] = int(training_set.loc[i,'toss_winner']==training_set.loc[i,'team1'])
    training_set.loc[i,'winner'] = int(training_set.loc[i,'winner']==training_set.loc[i,'team1'])

In [334]:
training_set = training_set.replace({'Chandigarh':'Mohali','Deccan Chargers':'Sunrisers Hyderabad', 'Pune Warriors':'Rising Pune Supergiant'})

In [335]:
training_set = training_set.dropna()
training_set

,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,...,month,day,runs1,runs2,wickets1,wickets2,fours1,fours2,sixes1,sixes2
id,,,,,,,,,,,,,,,,,,,,,
1,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,0,1.0,1,0,1.0,35,...,0.333333,0.166667,207.0,172.0,4.0,10.0,17.0,15.0,9.0,8.0
2,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,0,1.0,1,0,0.0,0,...,0.333333,0.200000,184.0,187.0,8.0,3.0,12.0,18.0,11.0,6.0
3,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,0,1.0,1,0,0.0,0,...,0.333333,0.233333,183.0,184.0,4.0,0.0,20.0,18.0,6.0,8.0
4,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,0,1.0,1,0,0.0,0,...,0.333333,0.266667,163.0,164.0,6.0,4.0,11.0,11.0,7.0,9.0
5,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,1,0.0,1,0,1.0,15,...,0.333333,0.266667,157.0,142.0,8.0,9.0,14.0,10.0,7.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,Raipur,2016-05-22,Delhi Daredevils,Royal Challengers Bangalore,0,1.0,1,0,0.0,0,...,0.416667,0.733333,138.0,139.0,8.0,4.0,9.0,13.0,3.0,2.0
633,Bangalore,2016-05-24,Gujarat Lions,Royal Challengers Bangalore,0,1.0,1,0,0.0,0,...,0.416667,0.800000,158.0,159.0,10.0,6.0,12.0,12.0,8.0,7.0
634,Delhi,2016-05-25,Sunrisers Hyderabad,Kolkata Knight Riders,0,1.0,1,0,1.0,22,...,0.416667,0.833333,162.0,140.0,8.0,8.0,14.0,8.0,7.0,3.0


In [336]:
test_set = pd.read_excel('test.xlsx')
test_set['toss_decision'] = (test_set['toss_decision']=='field').astype(int)
for i in range(120):
    test_set.loc[i,'month'] = test_set.loc[i,'date'].month/12
    test_set.loc[i,'day'] = test_set['date'][i].day/30
test_set = test_set.loc[:,['season','id','city','team1','team2','toss_winner','toss_decision','month','day']]
test_set = test_set.set_index('id')
for i in range(120):
    test_set.iloc[i,4] = int(test_set.iloc[i,4]==test_set.iloc[i,2])
test_set

,season,city,team1,team2,toss_winner,toss_decision,month,day
id,,,,,,,,
7894,2018,Mumbai,Mumbai Indians,Chennai Super Kings,0,1,0.333333,0.233333
7895,2018,Mohali,Delhi Daredevils,Kings XI Punjab,0,1,0.333333,0.266667
7896,2018,Kolkata,Royal Challengers Bangalore,Kolkata Knight Riders,0,1,0.333333,0.266667
7897,2018,Hyderabad,Rajasthan Royals,Sunrisers Hyderabad,0,1,0.333333,0.300000
7898,2018,Chennai,Kolkata Knight Riders,Chennai Super Kings,0,1,0.333333,0.333333
...,...,...,...,...,...,...,...,...
11347,2019,Mumbai,Kolkata Knight Riders,Mumbai Indians,0,1,0.416667,0.166667
11412,2019,Chennai,Chennai Super Kings,Mumbai Indians,1,0,0.416667,0.233333
11413,2019,Visakhapatnam,Sunrisers Hyderabad,Delhi Daredevils,0,1,0.416667,0.266667


In [337]:
t = pd.concat([training_set.loc[:,['city','team1','team2','toss_winner','toss_decision','month','day']],test_set.iloc[:,1:]],ignore_index=True)
encoder = LabelEncoder()
t['city'] = encoder.fit_transform(t['city'])
t['team1'] = encoder.fit_transform(t['team1'])
t['team2'] = encoder.fit_transform(t['team2'])
t.iloc[:626,-1] = encoder.fit_transform(t.iloc[:626,-1])
training_x = np.array(t.iloc[:627,:]).astype(float)
test_x = np.array(t.iloc[627:747,:]).astype(float)
training_y = np.array(training_set.loc[:,'winner':'win_by_wickets']).astype(float)

In [338]:
batsmen_win=training_y[:,1].sum()/training_set[training_set['win_by_runs']>0].count()[0]
training_y[:,1] /= 50
training_y[:,2] /= 10
training_y

array([[1.  , 0.7 , 0.  ],
       [0.  , 0.  , 0.7 ],
       [0.  , 0.  , 1.  ],
       ...,
       [1.  , 0.44, 0.  ],
       [0.  , 0.  , 0.4 ],
       [1.  , 0.16, 0.  ]])

In [ ]:
model = keras.Sequential([keras.layers.Dense(100,activation='relu'),
                          keras.layers.Dense(32,activation='relu'),
                          keras.layers.Dense(16,activation='relu'),
                         keras.layers.Dense(3,activation='sigmoid')])
model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])
model.fit(training_x,training_y,epochs=500)

Epoch 1/500
20/20 [==============================] - 2s 4ms/step - loss: 0.2761 - accuracy: 0.4662
Epoch 2/500
20/20 [==============================] - 0s 3ms/step - loss: 0.2149 - accuracy: 0.4366
Epoch 3/500
20/20 [==============================] - 0s 3ms/step - loss: 0.1914 - accuracy: 0.4739
Epoch 4/500
20/20 [==============================] - 0s 3ms/step - loss: 0.2067 - accuracy: 0.3953
Epoch 5/500
20/20 [==============================] - 0s 4ms/step - loss: 0.2004 - accuracy: 0.4316
Epoch 6/500
20/20 [==============================] - 0s 3ms/step - loss: 0.1892 - accuracy: 0.4656
Epoch 7/500
20/20 [==============================] - 0s 3ms/step - loss: 0.1978 - accuracy: 0.4559
Epoch 8/500
20/20 [==============================] - 0s 3ms/step - loss: 0.2058 - accuracy: 0.4703
Epoch 9/500
20/20 [==============================] - 0s 3ms/step - loss: 0.1945 - accuracy: 0.4291
Epoch 10/500
20/20 [==============================] - 0s 4ms/step - loss: 0.1993 - accuracy: 0.4457
Epoch 11/

20/20 [==============================] - 0s 2ms/step - loss: 0.1728 - accuracy: 0.6242
Epoch 83/500
20/20 [==============================] - 0s 2ms/step - loss: 0.1659 - accuracy: 0.5599
Epoch 84/500
20/20 [==============================] - 0s 2ms/step - loss: 0.1686 - accuracy: 0.6334
Epoch 85/500
20/20 [==============================] - 0s 2ms/step - loss: 0.1735 - accuracy: 0.5569
Epoch 86/500
20/20 [==============================] - 0s 3ms/step - loss: 0.1751 - accuracy: 0.5365
Epoch 87/500
20/20 [==============================] - 0s 2ms/step - loss: 0.1609 - accuracy: 0.5628
Epoch 88/500
20/20 [==============================] - 0s 2ms/step - loss: 0.1738 - accuracy: 0.5833
Epoch 89/500
20/20 [==============================] - 0s 3ms/step - loss: 0.1581 - accuracy: 0.6153
Epoch 90/500
20/20 [==============================] - 0s 3ms/step - loss: 0.1849 - accuracy: 0.5829
Epoch 91/500
20/20 [==============================] - 0s 3ms/step - loss: 0.1682 - accuracy: 0.6210
Epoch 92/500


In [ ]:
test_y = model.predict(test_x)
test_y[:,1] *= 50
test_y[:,2] *= 10
test_y = test_y.round()

In [ ]:
result = test_set
result['winner'] = test_y[:,0]
result['win_by_runs'] = test_y[:,1]
result['win_by_wickets'] = test_y[:,2]
for i in range(120):
    result.iloc[i,8] = result.iloc[i,2] if result.iloc[i,8] else result.iloc[i,3]
season1 = result[result['season']==2018]
season1 = season1.iloc[:-4,:]
season2 = result[result['season']==2019]
season2 = season2.iloc[:-4,:]
season1

In [ ]:
season2